In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.0-preview-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('International_Trip_mode').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
spark

In [4]:
#Import Library
#import findspark
#findspark.init()
#import pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
from pyspark.sql.functions import month
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.functions import round,array,col,ntile,rank,asc,sum,desc,lag,lead,when,first,min,max,count
# For Adding row number/index
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

from pyspark import SparkContext
from pyspark import SparkContext, SparkConf, keyword_only

#from pyspark.sql import SQLContext, HiveContext, SparkSession
from pyspark.sql import Row

In [5]:
trip_level2=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\international_trip_nov\*.csv",inferSchema='true')

In [6]:
trip_level2=trip_level2.withColumn('imsi_trip_id', concat(col('imsi'),lit('_'),col('trip_id')))

In [7]:
#trip_level2.show(2)

In [8]:
#trip_level2.toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_level_v2.csv")

In [9]:
#pa=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\PA.csv",inferSchema='true')
poi=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\POI.csv",inferSchema='true')

In [10]:
pa_data_all =spark.read.json(r"C:\Users\harin\Desktop\international\Raw_Data\PA.geojson").drop('_corrupt_record').dropna()

In [11]:
from pyspark.sql.functions import split,explode,regexp_replace,concat,window
win_long = Window.partitionBy('pa_id','pa_centroid')
pa_data_all1 = pa_data_all.withColumn('geometry1',col('geometry.coordinates')[0])\
.withColumn('geometry1',explode(col('geometry1')))\
.withColumn('pa_id',col('properties.pa_id'))\
.withColumn('pa_centroid',col('properties.pa_centroid')).select('pa_id','geometry1','pa_centroid')\
.withColumn('pa_long',col('geometry1')[0]).withColumn('pa_lat',col('geometry1')[1])

pa_data_max = pa_data_all1.withColumn('max_long',max('pa_long').over(win_long)).filter(col('max_long')==col('pa_long')).dropDuplicates()\
.withColumnRenamed('geometry1','pa_max').withColumnRenamed('pa_lat','max_lat').select('pa_id','pa_centroid','pa_max','max_long','max_lat')
pa_data_min = pa_data_all1.withColumn('min_long',min('pa_long').over(win_long)).filter(col('min_long')==col('pa_long')).dropDuplicates()\
.withColumnRenamed('geometry1','pa_min').withColumnRenamed('pa_lat','min_lat').select('pa_id','pa_centroid','pa_min','min_long','min_lat')

#pa_data_max.filter(col('pa_id')=='P00000018030').show(30,False)
#pa_data_min.filter(col('pa_id')=='P00000018030').show(30,False)

In [12]:
pa_data_all.createOrReplaceTempView("patable")
#pa_data_all.printSchema()
from shapely.geometry import Point, Polygon
padf = pa_data_max.join(pa_data_min,['pa_id','pa_centroid']).dropDuplicates()\
.withColumn('pa_centroid1',regexp_replace(col('pa_centroid'),'POINT',''))\
.withColumn("cenlong_pa", split(col("pa_centroid1"), " ").getItem(1)).withColumn("cenlat_pa", split(col("pa_centroid1"), " ").getItem(2))\
.withColumn('cenlong_pa',regexp_replace(col('cenlong_pa'),'\\(',''))\
.withColumn('cenlat_pa',regexp_replace(col('cenlat_pa'),'\\)','')).drop('pa_centroid1','pa_max','pa_min')
pa_id = padf.select('pa_id','max_long','max_lat','min_long','min_lat','cenlong_pa','cenlat_pa')
#pa_id.show(3,False)

In [13]:
#poi.columns

In [14]:
pa_id1 = pa_id.select("max_long","max_lat","pa_id").distinct()
pa_id2 = pa_id.select("min_long","min_lat","pa_id").distinct()
pa_id3 = pa_id.select("cenlong_pa","cenlat_pa","pa_id").distinct()
poi_id1 =poi.select('lm_id','lm_latitude','lm_longtitude','Mode').distinct()

In [15]:
max_arr = pa_id1.toPandas()
max_arr1 = max_arr[['max_long', 'max_lat','pa_id']].values.tolist()
min_arr = pa_id2.toPandas()
min_arr1 = min_arr[['min_long', 'min_lat','pa_id']].values.tolist()
cen_arr = pa_id3.toPandas()
cen_arr1 = cen_arr[["cenlong_pa","cenlat_pa","pa_id"]].values.tolist()
poi_arr = poi_id1.toPandas()
poi_arr1 = poi_arr[["lm_longtitude","lm_latitude","lm_id"]].values.tolist()
import numpy as np
import pandas as pd 
from scipy.spatial import cKDTree
def ckdnearest(points):
    k=1
    nA = pd.DataFrame(points,columns=["long","lat"]).values.tolist()
    btree = cKDTree(nA)
    def Extract(lst):
      return [(item[0],item[1]) for item in lst]
    #max_arr1 = pd.DataFrame(max_arr,columns=['long','lat','pa_id'])
    #max_arr1 = pa_id1.toPandas()
    max_arr2 = Extract(max_arr1)
    maxdist,idx = btree.query(max_arr2, k=1)
    mindist_index1 = np.argmin(maxdist)
    dist_arr1 = max_arr1[mindist_index1],maxdist[mindist_index1]
    #return np.float(round(maxdist[mindist_index1]),2)
    
    #min_arr1 = pd.DataFrame(min_arr,columns=['long','lat','pa_id'])
    #min_arr1 = pa_id2.toPandas()
    min_arr2 = Extract(min_arr1)
    mindist,idx = btree.query(min_arr2, k=1)
    mindist_index2 = np.argmin(mindist)
    dist_arr2 = min_arr1[mindist_index2],mindist[mindist_index2]
    
    #cen_arr1 = pd.DataFrame(cen_arr,columns=['long','lat','pa_id'])
    #cen_arr1 = pa_id3.toPandas()
    cen_arr2 = Extract(cen_arr1)
    cendist,idx = btree.query(cen_arr2, k=1)
    mindist_index3 = np.argmin(cendist)
    dist_arr3 = cen_arr1[mindist_index3],cendist[mindist_index3]
    
    
    zipped = [maxdist[mindist_index1]*111, mindist[mindist_index2]*111,cendist[mindist_index3]*111]
    zipped1 = [max_arr1[mindist_index1],min_arr1[mindist_index2],cen_arr1[mindist_index3]]
    df = pd.DataFrame(zipped1, columns=['long','lat','pa_id'])#,'local_cat'
    df['distance'] =(zipped)
    df1 = (df[df['distance'] == df['distance'].min()])[['pa_id','distance']]#,'local_cat'
    
    #poi_arr1 = pd.DataFrame(poi_arr,columns=['long','lat','lm_id'])
    #poi_arr1 = poi_id1.toPandas()
    poi_arr2 = Extract(poi_arr1)
    poidist,idx = btree.query(poi_arr2, k=1)
    mindist_index4 = np.argmin(poidist)
    dist_arr4 = poi_arr1[mindist_index4],poidist[mindist_index4]
    df_poi = pd.DataFrame([poi_arr1[mindist_index4]], columns=['long','lat','lm_id'])#,'lm_name','lm_sub_name'
    df_poi['distance']=(poidist[mindist_index4]*111)
    
    if  (df1['distance'].values[0]<=5.0):
        return  df1[['pa_id']].values.tolist()#df1[['local_cat']].apply(lambda x: str(x))
    elif (df_poi['distance'].values[0]<=5.0):
        return  df_poi[['lm_id']].values.tolist()
    else :
        return ['no category']
    

In [16]:
#df1.head()

In [17]:
#trip_level2.columns

In [18]:
from pyspark.sql.functions import concat,col,concat_ws

exit_df1 = trip_level2.select('imsi_trip_id','exit_grid_longitude','exit_grid_latitude')


exit_df2 = exit_df1.select('imsi_trip_id','exit_grid_longitude','exit_grid_latitude')\
.dropDuplicates().rdd.map(lambda x: (x[0], ([x[1], x[2]]))).groupByKey().mapValues(lambda _: ckdnearest((list(_))))

#print(exit_df1.count())
#print(clusterdf1.distinct().count())

In [19]:
from pyspark.sql.types import StructType,StructField, StringType,ArrayType
deptSchema = StructType([       
    StructField('imsi_trip_id', StringType(), True),
    StructField('features', ArrayType(StringType()), True)])

exit_df3 = spark.createDataFrame(exit_df2, schema = deptSchema).withColumn("exit_Pa_Poi", col("features").getItem(0).cast('String')).drop('features').dropDuplicates()
#exit_df3 = exit_df3.filter(col('exit_Pa_Poi')!='no category')
#exit_df3 = exit_df3.withColumn('exit_Pa_Poi',regexp_replace(col('exit_Pa_Poi'),'[','')).withColumn('exit_Pa_Poi',regexp_replace(col('exit_Pa_Poi'),']',''))
exit_df3.printSchema()
#exit_df3.show(truncate=False)

root
 |-- imsi_trip_id: string (nullable = true)
 |-- exit_Pa_Poi: string (nullable = true)



In [20]:
#exit_df3 = exit_df3.withColumn('exit_Pa_Poi',regexp_replace(col('exit_Pa_Poi'),'[','')).withColumn('exit_Pa_Poi',regexp_replace(col('exit_Pa_Poi'),']',''))
#exit_df3.show(12)

In [21]:
entry_df1 = trip_level2.select('imsi_trip_id','entry_grid_longitude','entry_grid_latitude')

entry_df2 = entry_df1.select('imsi_trip_id','entry_grid_longitude','entry_grid_latitude')\
.dropDuplicates().rdd.map(lambda x: (x[0], ([x[1], x[2]]))).groupByKey().mapValues(lambda _: ckdnearest((list(_))))

#print(entry_df1.count())

In [22]:
deptSchema = StructType([       
    StructField('imsi_trip_id', StringType(), True),
    StructField('features', ArrayType(StringType()), True)])

entry_df3 = spark.createDataFrame(entry_df2, schema = deptSchema).withColumn("entry_Pa_Poi", col("features").getItem(0).cast('String')).drop('features').dropDuplicates()
#entry_df3 = entry_df3.filter(col('entry_Pa_Poi')!='no category')
#entry_df3 = entry_df3.withColumn('entry_Pa_Poi',regexp_replace(col('entry_Pa_Poi'),'[','')).withColumn('entry_Pa_Poi',regexp_replace(col('entry_Pa_Poi'),']',''))
entry_df3.printSchema()
#entry_df3.show(truncate=False)

root
 |-- imsi_trip_id: string (nullable = true)
 |-- entry_Pa_Poi: string (nullable = true)



In [23]:
mode_data=exit_df3.join(entry_df3,['imsi_trip_id'],'outer')

In [24]:
from pyspark.sql import functions as F
mode_data = mode_data.withColumn('entry_Pa_Poi',F.regexp_replace("entry_Pa_Poi", "\\[", ""))
mode_data = mode_data.withColumn('entry_Pa_Poi',F.regexp_replace("entry_Pa_Poi", "\\]", ""))
mode_data = mode_data.withColumn('exit_Pa_Poi',F.regexp_replace("exit_Pa_Poi", "\\[", ""))
mode_data = mode_data.withColumn('exit_Pa_Poi',F.regexp_replace("exit_Pa_Poi", "\\]", ""))

In [25]:
#mode_data.count()

In [26]:
#mode_data.show()

In [27]:
pa_poi_mode=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\Pa_poi_mode.csv",inferSchema='true')

In [28]:
mode_data1=mode_data.join(pa_poi_mode,(mode_data.exit_Pa_Poi==pa_poi_mode.pa_id),'left').withColumnRenamed('Mode','exit_Mode').distinct()\
.select('imsi_trip_id','exit_Pa_Poi','entry_Pa_Poi','exit_Mode') 


In [29]:
mode_data2=mode_data1.join(pa_poi_mode,(mode_data1.entry_Pa_Poi==pa_poi_mode.pa_id),'left').withColumnRenamed('Mode','entry_Mode').distinct()\
.select('imsi_trip_id','exit_Mode','entry_Mode')

#mode_data2.show()

In [30]:
trip_mode=trip_level2.join(mode_data2,['imsi_trip_id'],'left').distinct()\
.select('imsi','trip_id','trip_start_time','trip_end_time','exit_grid_id','exit_grid_name','entry_grid_id',
 'entry_grid_name','exit_Mode','entry_Mode')

In [31]:
trip_mode.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_mode_nov")

In [32]:
spark.stop()